## 7.VGG模型（2014年）

**学习目标**

1. 理解为什么可以用3x3的卷积核来代替更大尺寸的卷积核

2. 会计算CNN网络的参数量

3. 熟悉模型参数的初始化方法

4. 会调节超参数：周期（epoch）、优化器的学习率（LR）、mini-batch

5. 理解批量归一化（BN）对训练深层网络的重要性

6. 会使用torch.utils.data.random_split划分数据集

****

7.1 VGGNet

VGG网络，全称为Visual Geometry Group网络，是一种在深度学习领域具有重要影响力的卷积神经网络（CNN）架构。它由牛津大学的视觉几何组（Visual Geometry Group）提出，并在2014年的ImageNet挑战赛中取得了优异的成绩。

7.2 VGG的关键技术

与AlexNet相比，VGG（Visual Geometry Group）网络的改进和优化：

（1）统一的3×3卷积核
VGGNet在所有卷积层中统一使用了3x3的卷积核，而AlexNet使用了不同大小的卷积核（如11x11、5x5等）。统一的3×3卷积核，能够在保证感受野大小的同时，减少模型的参数量。同时，由于3×3卷积核可以看作是一种特殊的1×1和5×5卷积核的组合，因此它能够在一定程度上模拟更大卷积核的效果，提高模型的表达能力。

（2）深度与性能的关系
VGGNet通过构建不同深度的网络结构，探索了卷积神经网络的深度与其性能之间的关系。实验结果表明，随着网络深度的增加，模型的性能也会相应提高。这为后续的研究提供了重要的启示，即构建更深的网络结构是提高模型性能的有效手段之一。

***

7.3 VGG16模型

1. VGG16模型结构
<img src="./images/VGG16.png" style="zoom:80%;" />

2. VGG16模型的参数量计算

（1） **卷积层参数**：每个卷积层的参数量由以下公式给出： 参数量=卷积核尺寸×输入通道数×输出通道数 

（2） **池化层参数**：池化层没有参数，因此它们的参数量为0。

（3） **全连接层参数**：每个全连接层的参数量由以下公式给出： 参数量=(前一层节点数+1)×本层节点数参数量=(前一层节点数+1)×本层节点数 
   其中“+1”是因为全连接层包含偏置项。VGG16的全连接层的节点数分别为4096, 4096, 和类别数（例如1000）。

（4） **偏置参数**：每个有偏置的层都有一个额外的偏置参数，其数量等于该层的输出通道数或节点数。

我们可以进行以下计算：

- 第一个卷积层的参数量：(3×3×3)×64=1728
  
  计算过程是这样：因为输入特征图（图像）数量为3，所以要用3个卷积核(尺寸为3x3）分别与输入特征图卷积得到3个图，然后将这三个图对应元素相加，得到一个输出特征图，因为有64个输出特征图，所以卷积核的数量就是3x64，总的参数量就是3x63x(3x3)。

- 后续卷积层的参数量：对于每个数量级的卷积层，参数量为 (3×3×𝐶in)×𝐶out，其中𝐶in是输入通道数，𝐶out是输出通道数。
- 全连接层的参数量：第一层全连接层的参数量为 7×7×512×4096+4096（加上4096是因为偏置项），第二层为 4096×4096+4096，最后一层为 4096×1000+1000。

<img src="./images/VGG16_params.png" style="zoom:100%;" />

将所有卷积层和全连接层的参数量加起来，就可以得到VGG16模型的总参数量。VGG16的总参数量大约是138M（即138,357,544个参数）。这个数字包括了所有的权重和偏置参数。

1. 感受野

在卷积神经网络中，两个3x3的卷积堆叠获得的感受野大小，相当一个5x5的卷积；3个3x3卷积的堆叠获取到的感受野相当于一个7x7的卷积。

卷积核的大小和数量是设计网络结构时需要考虑的重要因素。使用2个3x3的卷积核代替1个5x5的卷积核有几个优势：

- **参数数量减少**：5x5的卷积核有5×5=255×5=25个参数，而两个3x3的卷积核总共有2×3×3=182×3×3=18个参数。这减少了参数的数量，有助于减少模型的复杂度和过拟合的风险。

- **增加非线性**：使用两个3x3的卷积核可以增加网络的非线性能力。每个卷积核后面通常跟着一个非线性激活函数（如ReLU）。两个连续的3x3卷积层相当于在网络中增加了额外的非线性变换，这有助于网络学习更复杂的特征。

- **感受野相同**：两个3x3卷积核堆叠起来，其有效感受野与单个5x5卷积核相同。第一个卷积核覆盖了3x3的区域，第二个卷积核由于第一个卷积的输出作为输入，可以覆盖到5x5的区域。这意味着网络可以在更大的范围内捕捉特征，而不需要增加卷积核的尺寸。

- **网络深度增加**：在VGG网络中，通过使用多个较小的卷积核，可以增加网络的深度。较深的网络能够学习更复杂的特征层次，这对于图像识别等任务是有益的。

- **保持特征图尺寸**：使用3x3卷积核可以在不改变特征图尺寸的情况下提取特征，这对于保持网络中特征的空间分辨率是有益的。而使用5x5卷积核可能会在每次卷积后减少特征图的尺寸，除非使用步长或填充策略来补偿。

- **计算效率**：较小的卷积核可以减少计算量，因为每次卷积操作涉及的元素更少。这可以提高网络的训练和推理速度。

- **更好的梯度流**：较小的卷积核有助于梯度在反向传播过程中更有效地流动，这有助于网络的训练，特别是在深层网络中。

所以使用两个3x3卷积核代替一个5x5卷积核是一种设计选择，它在减少参数数量、增加网络深度和非线性能力、保持感受野和计算效率方面提供了平衡。这种设计在VGG网络中被证明是有效的，并在深度学习领域得到了广泛应用。



7.3 基于VGG16的土豆疾病识别

<img src="./images/PotatoPlantDiseases.jpg" style="zoom:100%;" />

1.导入必需的模块

In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torchvision
from torchvision import datasets, transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

2.数据集的加载和预处理

In [3]:
# 定义转换操作
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [4]:
# 加载数据集
image_dataset = datasets.ImageFolder('./datasets/Potato Plant Diseases/PotatoPlants', train_transform)
len(image_dataset)

2152

In [5]:
# 划分数据集
split_size = int(0.8 * len(image_dataset))
train_dataset, val_dataset = random_split(image_dataset, [split_size, len(image_dataset) - split_size])

# 为测试集设置变换操作
val_dataset.dataset.transform = val_transform

In [6]:
# 创建DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

3.模型构建

In [7]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),  # BN
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),  # BN
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),  # BN
            nn.ReLU(inplace=True),
            
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),  # BN
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),  # BN
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),  # BN
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),  # BN
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # BN
            nn.ReLU(inplace=True),
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # BN
            nn.ReLU(inplace=True),
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # BN
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Block 5
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # BN
            nn.ReLU(inplace=True),
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # BN
            nn.ReLU(inplace=True),
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # BN
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            # nn.BatchNorm1d(4096),  # 1D BN
            nn.ReLU(inplace=True),
            nn.Dropout(),
            
            nn.Linear(4096, 4096),
            # nn.BatchNorm1d(4096),  # 1D BN
            nn.ReLU(inplace=True),
            nn.Dropout(),
            
            nn.Linear(4096, 3)  # 输出层，3分类任务
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # 展开特征图
        x = self.classifier(x)
        return x

In [8]:
model = VGG16().to(device)
print(model)

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 2

In [9]:
# 定义超参数
batch_size = 64
learning_rate = 0.01
num_epochs = 10

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

4.模型参数的初始化

如果你没有手动初始化模型参数，PyTorch会使用其默认的参数初始化方法。对于不同的层和参数类型，PyTorch有不同的默认初始化策略：

（1）权重（Weights）

对于大多数线性层（nn.Linear）和卷积层（nn.Conv2d），如果没有指定初始化方法，PyTorch默认使用Kaiming He初始化（也称为He初始化），这是一种基于输入特征的初始化方法，适用于ReLU激活函数。

（2）偏置（Bias）

对于偏置参数，PyTorch默认将其初始化为0。

（3）BatchNorm的参数

对于批量归一化层（nn.BatchNorm），如果没有指定初始化方法，PyTorch默认将γ（缩放参数）初始化为1，将β（偏移参数）初始化为0。

（4）其他层

对于其他类型的层，如循环层（nn.LSTM、nn.GRU）等，PyTorch也有自己的默认初始化策略，通常是将权重初始化为较小的随机值，偏置初始化为0。

默认初始化通常是一个好的起点，但根据你的具体任务和网络结构，可能需要进行自定义初始化以获得更好的性能。例如，如果你使用的是Sigmoid或Tanh激活函数，可能需要使用Xavier初始化（也称为Glorot初始化）。

4.模型训练

In [10]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        # # 打印每个批次的进度和损失
        # if (i + 1) % 10 == 0:  # 每10个批次打印一次
        #     print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}")
        
    # 计算并打印平均损失
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss}")

torch.save(model,'weights/vgg16.pth')
# torch.cuda.empty_cache()

Epoch [1/10] Loss: 0.5552565852801005
Epoch [2/10] Loss: 0.4548258886293129
Epoch [3/10] Loss: 0.2999280255149912
Epoch [4/10] Loss: 0.25158017059719123
Epoch [5/10] Loss: 0.1689667705860403
Epoch [6/10] Loss: 0.16401880427643103
Epoch [7/10] Loss: 0.14885671119447108
Epoch [8/10] Loss: 0.14012972913958407
Epoch [9/10] Loss: 0.15228749107983378
Epoch [10/10] Loss: 0.10191185731026861


5.模型测试

In [11]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f"Accuracy: {(correct / total) * 100:.2f}%")

Accuracy: 94.20%


**批量归一化（Batch Normalization）的作用**

训练深层神经⽹络是⼗分困难的，特别是在较短的时间内使他们收敛更加棘⼿。批量归一化（Batch Normalization, BN）最早由Sergey Ioffe和Christian Szegedy在2015年提出。这种技术被设计用来解决深度神经网络训练中的内部协变量偏移问题，并加速模型的收敛速度。BN的引入显著提高了深度神经网络的训练效率，并有助于提高模型的泛化能力，成为深度学习训练过程中的标准组件之一。

⾸先，数据预处理的⽅式通常会对最终结果产⽣巨⼤影响。使⽤真实数据时，我们的第⼀步是标准化输⼊特征，使其平均值为0，⽅差为1。直观地说，这种标准化可以很好地与我们的优化器配合使⽤，因为它可以将参数的量级进⾏统⼀。

第⼆，对于典型的多层感知机或卷积神经⽹络。当我们训练时，中间层中的变量可能具有更⼴的变化范围：不论是沿着从输⼊到输出的层，跨同⼀层中的单元，或是随着时间的推移，模型参数的随着训练更新变幻莫测。批量标准化的发明者⾮正式地假设，这些变量分布中的这种偏移可能会阻碍⽹络的收敛。直观地说，我们可能会猜想，如果⼀个层的可变值是另⼀层的 100 倍，这可能需要对学习率进⾏补偿调整。

第三，更深层的⽹络很复杂，容易过拟合。这意味着正则化变得更加重要。批量标准化应⽤于单个可选层（也可以应⽤到所有层），其原理如下：在每次训练迭代中，我们⾸先归⼀化输⼊，即通过减去其均值并除以其标准差，其中两者均基于当前小批量处理。接下来，我们应⽤⽐例系数和⽐例偏移。正是由于这个基于批量统计的标准化，才有了批量标准化的名称。

总结起来批量归一化的作用有以下三点：

（1）加快网络训练的收敛速度；

（2）控制梯度爆炸/防止梯度消失；

（3）防止过拟合。